# 🧠 3LC x AWS Cloud @ UT Dallas Hackathon
## Chihuahua vs Muffin Classification Challenge

**Welcome!** This starter notebook will help you:
1. Load and explore the dataset
2. Train a baseline CNN model (~83% accuracy)
3. Integrate with 3LC for data-centric AI
4. Learn the Train-Fix-Retrain workflow

---

### 📋 Your Task
Use data-centric AI techniques to **improve model accuracy** beyond the baseline. Good luck! 🚀


## 1. Setup and Imports


In [ ]:
# Standard libraries
import os
import sys
from pathlib import Path
import random

# Data science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

# Image processing
from PIL import Image

# Progress bars
from tqdm.notebook import tqdm

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 2. Configure Paths and Hyperparameters


In [ ]:
# Project paths (relative to notebook location)
BASE_PATH = Path('../')
DATA_PATH = BASE_PATH / 'data'
TRAIN_PATH = DATA_PATH / 'train'
TEST_PATH = DATA_PATH / 'test'

# Hyperparameters
BATCH_SIZE = 16
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
IMG_SIZE = 128

# Display paths
print("📂 Project Structure:")
print(f"  Base:  {BASE_PATH.absolute()}")
print(f"  Data:  {DATA_PATH.absolute()}")
print(f"  Train: {TRAIN_PATH.absolute()}")
print(f"  Test:  {TEST_PATH.absolute()}")

# Check if data exists
if TRAIN_PATH.exists():
    print("\n✅ Training data found!")
else:
    print("\n❌ Training data not found! Please download the dataset.")


## 3. Load and Explore Dataset


In [ ]:
# Data transformations
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=str(TRAIN_PATH), transform=train_transform)

# Split into train/validation (80/20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], 
                                          generator=torch.Generator().manual_seed(SEED))

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"📦 Dataset Split:")
print(f"   Training:   {len(train_dataset)} samples")
print(f"   Validation: {len(val_dataset)} samples")
print(f"   Classes: {full_dataset.classes}")


## 4. Define Baseline CNN Model


In [ ]:
class SimpleCNN(nn.Module):
    """Baseline CNN for binary classification"""
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Create model
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

total_params = sum(p.numel() for p in model.parameters())
print(f"🧠 Model created with {total_params:,} parameters")


## 5. Training Loop


In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Validating", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total

print("✅ Training functions defined")


### Run Training


In [ ]:
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0

print("🚀 Starting training...\n" + "="*60)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch [{epoch+1}/{NUM_EPOCHS}]")
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # Create models directory if it doesn't exist
        os.makedirs('../models', exist_ok=True)
        torch.save({'model_state_dict': model.state_dict(), 'val_acc': val_acc}, 
                  '../models/best_model.pth')
        print(f"  ✅ New best model saved! (Val Acc: {val_acc:.2f}%)")

print(f"\n{'='*60}\n✅ Training complete! Best validation accuracy: {best_val_acc:.2f}%")


## 6. Visualize Results


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(history['train_loss'], label='Train Loss', marker='o')
ax1.plot(history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history['train_acc'], label='Train Accuracy', marker='o')
ax2.plot(history['val_acc'], label='Val Accuracy', marker='s')
ax2.axhline(y=83, color='r', linestyle='--', label='Baseline (83%)')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Final Results:")
print(f"   Best Validation Accuracy: {best_val_acc:.2f}%")
if best_val_acc > 83:
    print(f"   ✅ You beat the baseline by {best_val_acc - 83:.2f}%!")
else:
    print(f"   💪 Try to improve by {83 - best_val_acc:.2f}% to beat baseline")


## 7. 3LC Integration (Advanced)

**Your Next Steps:**
1. Install 3LC: `pip install tlc`
2. Sign up at [3lc.ai](https://3lc.ai)
3. Register your dataset as a 3LC Table
4. Extract embeddings from your model
5. Use 3LC Dashboard to explore and identify problematic samples
6. Apply data-centric improvements (remove outliers, fix labels, etc.)
7. **Retrain and iterate!**

### Train-Fix-Retrain Loop:
- **Analyze:** Use 3LC to find misclassified/ambiguous samples
- **Fix:** Remove bad data, add augmentation, rebalance classes
- **Retrain:** Train on improved dataset
- **Compare:** Track improvements in 3LC table revisions

---

## 8. Your Turn to Improve! 🎯

### Ideas:
1. **Use Transfer Learning:** Try ResNet18 or EfficientNet
2. **Better Augmentation:** Add more aggressive transforms
3. **Data-Centric:** Use 3LC to identify and fix data issues
4. **Hyperparameter Tuning:** Adjust learning rate, batch size, etc.

**Goal:** Beat 83% baseline accuracy using data-centric AI!


In [ ]:
# YOUR CODE HERE - Experiment and improve!
# Example: Try transfer learning with ResNet

# from torchvision.models import resnet18
# model_improved = resnet18(pretrained=True)
# model_improved.fc = nn.Linear(model_improved.fc.in_features, 2)
# model_improved = model_improved.to(device)
# ... train with improved model

print("💡 Add your improvements here and retrain!")
print("Remember to use 3LC for data-centric insights!")
